In [ ]:
from langgraph.graph import StateGraph , START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict , Annotated
from dotenv import load_dotenv 
from pydantic import BaseModel, Field
import os
import operator

In [ ]:
load_dotenv()  # take environment variables from .env

llm = ChatOpenAI(
    model="gpt-4o-mini",                # ✅ just the plain name
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

In [ ]:
class EvaluationSchema(BaseModel):
    score: int = Field(..., description="Score from 1 to 10" , ge=0 , le=10)
    feedback: str = Field(..., description="Detailed feedback for the essay")

In [ ]:
structured_model = llm.with_structured_output(EvaluationSchema)

In [ ]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    clarity_feedback: str
    analysis_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int],operator.add]
    avg_score: float

In [ ]:
def evaluate_language(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the following essay for language quality, grammar, and vocabulary:\n\n{state['essay']}\n\nProvide a score from 1 to 10 and detailed feedback."
    result = structured_model.invoke(prompt)

    return {'language_feedback': result.feedback , 'individual_scores': [result.score]}

In [ ]:
def evaluate_analysis(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the following essay for depth of analysis, argument strength, and evidence:\n\n{state['essay']}\n\nProvide a score from 1 to 10 and detailed feedback."
    result = structured_model.invoke(prompt)

    return {'analysis_feedback': result.feedback , 'individual_scores': [result.score]}

In [ ]:
def evaluate_clarity(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the following essay for clarity, coherence, and structure:\n\n{state['essay']}\n\nProvide a score from 1 to 10 and detailed feedback."
    result = structured_model.invoke(prompt)

    return {'clarity_feedback': result.feedback , 'individual_scores': [result.score]}

In [ ]:
def final_evaluation(state: UPSCState) -> UPSCState:
    prompt = f"Based on the following feedbacks:\n\nLanguage Feedback: {state['language_feedback']}\nClarity Feedback: {state['clarity_feedback']}\nAnalysis Feedback: {state['analysis_feedback']}\n\nProvide an overall evaluation of the essay, summarizing strengths and areas for improvement."
    overall_feedback = llm.invoke(prompt).content
    avg_score = sum(state['individual_scores']) / len(state['individual_scores'])

    return {'overall_feedback': overall_feedback , 'avg_score': avg_score}

In [ ]:
graph = StateGraph(UPSCState)
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_clarity', evaluate_clarity)
graph.add_node('final_evaluation', final_evaluation)

graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis') 
graph.add_edge(START, 'evaluate_clarity')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_clarity', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [ ]:
initial_state = {
    'essay' : """climate change is a big problam in the world today and its making everything worst then before. People always saying that it not real but it definetly is real becaus the weather is acting very strange and crazy in many country. Sometime it rain too much and sometime no rain at all for long time and farmer can not grow crops proper. This is danger for food secuirty but still many goverments dont care or just ignore until is too late.

Also animal suffer a lot from this changes, like polar bears who dont have no ice to live on and they starve hungry. And birds dont know when to migrate cause the season mix up, spring come late but also early sometimes. Its confusing. Human people also feel it, like heatwave kill many persons in city, and old people and babys get sick more. The sea level also is rising, and some island might be disappear under water completly, imagine that happen to ur home!

People say solution is planting trees or make solar panel but not enuff action is doing. Big companys pollute much and then pretend like nothing is wrong. Politician make meeting and talking talking but very less happen. Some rich country blame poor country and poor blame rich, so round and round circle, no real fix come. If we keep like this then future generation gonna hate us becaus we leave them broken planet.

So climate change is real and very danger, it need urgent action immediatly. People need to think about planet not just money money money. If not, the earth will be more hot and bad and no one can survive good."""}

result = workflow.invoke(initial_state)

print(f"Individual Score : {result['individual_scores']}")
print(f"Average Score : {result['avg_score']}")
print(f"Overall Feedback : {result['overall_feedback']}")



In [ ]:
workflow